In [ ]:
# Подготовка по хакатону

In [ ]:
# # Пример запуска в поточном режиме

# from threading import Thread
# from time import sleep
# def func():
#     for i in range(5):
#         print(f"from child thread: {i}")
#         sleep(0.5)
# th = Thread(target=func)
# th.start()
# for i in range(5):
#     print(f"from main thread: {i}")
#     sleep(1)

In [ ]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 8.2 MB 10.7 MB/s 


In [ ]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
! pip install thread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement thread (from versions: none)
ERROR: No matching distribution found for thread


In [3]:
import pandas as pd
import pymorphy2
import nltk
import re

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import datetime
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
from gensim.models import *
from gensim import corpora
from threading import Thread
from time import sleep

# from google.colab import drive

In [ ]:
# dict_themes = {
#   'Психология': ['психология', 'когнитивный', 'мышление', 'установка', 'ограничение', 'развитие', 'мотивация', 'человек'],
#   'Программирование': ['программа', 'программирование', 'приложение']
# }

# dict_themes['Психология']

dir_hackathon = '/content/gdrive/MyDrive/Colab Notebooks/Hackathon/'
dir_data_in = f'{dir_hackathon}/Data/In'
dir_data_out = f'{dir_hackathon}/Data/Out'

column_num = 'Номер'
column_name = 'Общее наименование продукции'
column_group = 'Группа продукции'

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
all_data_in = pd.read_excel(f'{dir_data_in}/dataset.xlsx', header=0)
all_data_in.columns = ['Номер продукции', 'Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции', 'Общее наименование продукции', 
                       'ИЛ', 'Заявитель', 'Адрес Заявителя', 'Изготовитель', 'Страна', 'Адрес изготовителя']
all_data_in.head(2)

,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя
0,37632 Продукция,4016999708,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...,ИЛ Мопалия Миддендорфа,ООО Европейский гологлаз 6,РОССИЯ 461220 БАРАБАНОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Ара Леара 11,РОССИЯ,РОССИЯ 182375 ЛУКИНО ПСКОВСКАЯ ОБЛАСТЬ
1,37565 Продукция,7308909809,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...,ИЛ Мопалия Миддендорфа,ООО Дроздовидная6,РОССИЯ 461182 ШЕСТАКОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Аннамский палочник 11,РОССИЯ,РОССИЯ 182306 ПОДДУБЬЕ ПСКОВСКАЯ ОБЛАСТЬ


In [ ]:
# data_name_group = all_data_in[['Группа продукции', 'Общее наименование продукции']]
data_name_group = all_data_in[[column_name]]
data_name_group

,Общее наименование продукции
0,Проходки кабельные модульные универсальные огн...
1,Ворота противопожарные металлические распашные...
2,Клапаны противопожарные марки КПВ.01(EI60) пря...
3,Вентиляторы крышные радиальные дымоудаления с ...
4,Вентиляторы радиальные дымоудаления типов ВР-8...
...,...
66748,Блок пробоотборный обогреваемый
66749,Система отбора проб природного газа
66750,Фильтры
66751,Элементы трубопроводов


In [ ]:
# prepare data:
def prepare_data(df_data):
  prepared_data = df_data
  for row in prepared_data.columns:
      prepared_data[row] = prepared_data[row].str.replace(r'\W', ' ')    
      prepared_data[row] = prepared_data[row].str.replace(r'\d', ' ')    
      prepared_data[row] = prepared_data[row].str.lower()
      prepared_data[row] = prepared_data[row].str.replace(r' +', ' ') 

  return df_data

prepared_data = prepare_data(data_name_group)
prepared_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,Общее наименование продукции
0,проходки кабельные модульные универсальные огн...
1,ворота противопожарные металлические распашные...
2,клапаны противопожарные марки кпв ei прямоугол...
3,вентиляторы крышные радиальные дымоудаления с ...
4,вентиляторы радиальные дымоудаления типов вр д...


In [ ]:
# Посмотреть, можеть добавить колонку с нумерацией

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = nltk.corpus.stopwords.words("russian")
stop_words.extend(['https', 'который', 'это', 'также', 'новый', 'являться', 'позволять', 'возможность', 
                   'наш', 'свой', 'ru', 'ещё', 'www', 'ru', 'com', 'пока', 'год'])

def lemmatize_cell(data_lemm, x, y):
    lemma = pymorphy2.MorphAnalyzer()
    cell = data_lemm.loc[y, data_lemm.columns[x]]
    print('Coords: ', x, y, '\n') #, datetime.datetime.now()
    text = ''
    if type(cell) == str and len(cell) > 0:
        text = " ".join([lemma.parse(word)[0].normal_form for word in cell.split(' ')])
        tokens = word_tokenize(text, language="russian")
#         tokens = lemma.parse(word)[0].normal_form 
        for token in tokens:
            if token in stop_words:
                tokens.remove(token)
        text = " ".join(tokens)
        
#     print(text, '\n')
        
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
 prepared_data['Номер'] = range(1, len(prepared_data) + 1)
 prepared_data = prepared_data.reindex(columns=['Номер', 'Группа продукции', 'Общее наименование продукции'])
 prepared_data.tail(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Номер,Группа продукции,Общее наименование продукции
66751,66752,элементы оборудования сборочные единицы и комп...,элементы трубопроводов
66752,66753,электрическое электрооборудование и неэлектрич...,станция газозаправочная автомобильная


In [ ]:
# def func():
#     for i in range(5):
#         print(f"from child thread: {i}")
#         sleep(0.5)
# th = Thread(target=func)
# th.start()
# for i in range(5):
#     print(f"from main thread: {i}")
#     sleep(1)

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
TypeError: lemmatize_cell() missing 3 required positional arguments: 'data_lemm', 'x', and 'y'



In [ ]:
%%time
data_lemm = prepared_data

data_lemm_out = pd.DataFrame()
data_lemm_out['Номер'] = data_lemm['Номер']
x_max = data_lemm.shape[1]
# x_max = 20
y_max = data_lemm.shape[0]
data_lemm_out = pd.DataFrame(columns=data_lemm.columns)

for x in range(1, x_max): # Прогон, без номера
    for y in range(y_max):  
        # th = Thread(target=lemmatize_cell(data_lemm, x, y))
        # th.start()

        cell = lemmatize_cell(data_lemm, x, y)
        data_lemm_out.loc[y, data_lemm_out.columns[x]] = cell
        
data_lemm_out

Выходные данные были обрезаны до нескольких последних строк (5000).
Coords:  2 56449 

Coords:  2 56450 

Coords:  2 56451 

Coords:  2 56452 

Coords:  2 56453 

Coords:  2 56454 

Coords:  2 56455 

Coords:  2 56456 

Coords:  2 56457 

Coords:  2 56458 

Coords:  2 56459 

Coords:  2 56460 

Coords:  2 56461 

Coords:  2 56462 

Coords:  2 56463 

Coords:  2 56464 

Coords:  2 56465 

Coords:  2 56466 

Coords:  2 56467 

Coords:  2 56468 

Coords:  2 56469 

Coords:  2 56470 

Coords:  2 56471 

Coords:  2 56472 

Coords:  2 56473 

Coords:  2 56474 

Coords:  2 56475 

Coords:  2 56476 

Coords:  2 56477 

Coords:  2 56478 

Coords:  2 56479 

Coords:  2 56480 

Coords:  2 56481 

Coords:  2 56482 

Coords:  2 56483 

Coords:  2 56484 

Coords:  2 56485 

Coords:  2 56486 

Coords:  2 56487 

Coords:  2 56488 

Coords:  2 56489 

Coords:  2 56490 

Coords:  2 56491 

Coords:  2 56492 

Coords:  2 56493 

Coords:  2 56494 

Coords:  2 56495 

Coords:  2 56496 

Coords:  2 56497 

C

KeyboardInterrupt: ignored

In [ ]:
data_lemm_out.head()

NameError: ignored

In [ ]:
data_lemm_out.to_excel(f'{dir_data_out}/data_lemm_morphy.xlsx', header=0)

NameError: ignored

In [1]:
data_lemm_out = pd.read_excel(f'{dir_data_out}/data_lemm_morphy.xlsx', header=0)
data_lemm_out.columns = [column_num, column_name, column_group]

data_lemm_out.tail(2)

NameError: name 'pd' is not defined

In [ ]:
data_lemm_out_small = data_lemm_out[:500]
data_lemm_out_small.tail(2)

,Номер,Общее наименование продукции,Группа продукции
498,499,игрушка электрический товар развлечение настол...,игрушка ребёнок старший год пластмасса
499,500,игрушка электрический товар развлечение настол...,игрушка ребёнок старший х игра настольный разв...


In [ ]:
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [4]:
data_lemm_out = pd.read_excel(f'{dir_data_out}/data_lemm_morphy.xlsx', header=0)
data_lemm_out.columns = [column_num, column_name, column_group]

data_lemm_out.tail(2)

NameError: name 'dir_data_out' is not defined

In [ ]:
column_id_group = 'Группа продукции_ordinal'

data_new = pd.read_excel(f'{dir_data_in}/dateset_categorical.xlsx', header=0)
data_claster_out_name = data_lemm_out[[column_num, column_name]]
data_claster_out_gr = data_new[[column_num, column_id_group]]
data_claster_out = pd.merge(data_claster_out_name, data_claster_out_gr, on="Номер")
data_claster_out.tail(3)

,Номер,Общее наименование продукции,Группа продукции_ordinal
66749,66750,электрический электрооборудование неэлектричес...,1939.0
66750,66751,элемент оборудование сборочный единица комплек...,2023.0
66751,66752,электрический электрооборудование неэлектричес...,1939.0


In [ ]:
data_claster_out[column_name] = data_claster_out[column_name].astype('str')
words = " ".join(data_claster_out[column_name]).split()
print(words[:1000])

['противопожарный', 'окно', 'дверь', 'дверь', 'шахта', 'лифт', 'нормировать', 'предел', 'огнестойкость', 'ворота', 'люк', 'штора', 'роллета', 'экран', 'занавес', 'клапан', 'противопожарный', 'нормально', 'открытый', 'клапан', 'противопожарный', 'нормально', 'закрытый', 'люк', 'дымовой', 'вытяжной', 'вентилятор', 'вытяжной', 'вентилятор', 'свитер', 'джемпер', 'жакет', 'юбка', 'брюки', 'костюм', 'рейтузы', 'полукомбинезон', 'комбинезон', 'шорты', 'платье', 'комплект', 'блузка', 'сорочка', 'верхний', 'жилет', 'аналогичный', 'изделие', 'заявить', 'изготовитель', 'предназначить', 'ребёнок', 'подросток', 'пижама', 'кальсоны', 'панталоны', 'фуфайка', 'комбинация', 'купальный', 'изделие', 'трус', 'майка', 'аналогичный', 'изделие', 'заявить', 'изготовитель', 'предназначить', 'ребёнок', 'старший', 'и', 'подросток', 'клапан', 'противопожарный', 'нормально', 'открытый', 'клапан', 'противопожарный', 'нормально', 'закрытый', 'люк', 'дымовой', 'клапан', 'противопожарный', 'нормально', 'открытый', 'кл

In [ ]:
# Common words in df
data_claster_out[column_name] = data_claster_out[column_name].astype('str')
words = " ".join(data_claster_out[column_name]).split()
print(words)
common_projects_words = Counter(words).most_common(1000)
common_projects_words

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[('изделие', 18964),
 ('предназначить', 15463),
 ('средство', 10530),
 ('оборудование', 9873),
 ('ребёнок', 8409),
 ('аналогичный', 8108),
 ('рабочий', 7994),
 ('заявить', 7930),
 ('изготовитель', 7930),
 ('взрывоопасный', 7912),
 ('подросток', 7852),
 ('свыше', 7355),
 ('среда', 7129),
 ('давление', 7094),
 ('газ', 6968),
 ('иметь', 6831),
 ('они', 6795),
 ('перевозка', 6325),
 ('оружие', 6305),
 ('назначение', 5771),
 ('категория', 5719),
 ('транспортный', 5669),
 ('допустимый', 5449),
 ('бельё', 5426),
 ('электрический', 5097),
 ('общий', 5096),
 ('игрушка', 5057),
 ('использовать', 4793),
 ('мпа', 4704),
 ('устройство', 4684),
 ('тот', 4572),
 ('число', 4572),
 ('максимально', 4564),
 ('нательный', 4474),
 ('костюм', 4447),
 ('нарезной', 4259),
 ('карабин', 4200),
 ('машина', 4166),
 ('другой', 4147),
 ('электрооборудование', 3944),
 ('морской', 3920),
 ('эксплуатация', 3906),
 ('ядерный', 3906),
 ('брюки', 3894),
 ('значение', 3890),
 ('мм', 3834),
 ('прочий', 3834),
 ('вместимост

In [ ]:
data_common_words = pd.DataFrame(list(common_projects_words))
data_common_words.columns = ['Слово', 'Количество ']
data_common_words.to_excel(f'{dir_data_out}/data_common_words.xlsx')
data_common_words

,Слово,Количество
0,изделие,18964
1,предназначить,15463
2,средство,10530
3,оборудование,9873
4,ребёнок,8409
...,...,...
995,разветвитель,55
996,переходник,55
997,подушка,55
998,опрокидывание,54


In [ ]:
words = ' '.join(data_common_words['Слово']).split()
# words
type(len(words)) 

int

In [ ]:
data_clasters_common_words[1:1, total_count_words]

TypeError: ignored

In [ ]:
# Get top  from clasters
def top_terms(df, top_n=50):
    words = ' '.join(df[column_name]).split()
    count_words = len(words)  
    list_top_words_and_count = Counter(words).most_common(top_n)
    # print(list_top_words_and_count)

    # list_top_words_count = [a_tuple[1] for a_tuple in list_top_words_and_count]
    # list_top_words = [a_tuple[0] for a_tuple in list_top_words_and_count]

    # data_claster_common_words = pd.DataFrame(list(common_projects_words))
    # arr = [list_top_words_and_count, count_words]

    # data_claster_common_words = pd.DataFrame((list_top_words_and_count))
    # columns=['Частые слова и их количество', 'Общее количество слов']
    
    # return data_claster_common_words # Не выгружается, слишком большой файл
    return list_top_words_and_count, count_words
    # return list_top_words_and_count

clasters_common_words = data_claster_out.groupby(column_id_group).apply(lambda x: top_terms(x, top_n=100))
count_clasters = len(clasters_common_words)
clasters_common_words

#  nums, words = map(list, zip(*nums_and_words)) 
words_info, count_words = map(list, zip(*clasters_common_words)) 
# data_clasters_common_words = pd.DataFrame([words_info, count_words], index=data_lemm_out[column_num], columns=['Частые слова и их количество', 'Общее количество слов'] )
frequent_words_count = 'Частые слова и их количество'
total_count_words = 'Общее количество слов'
data_clasters_common_words = pd.DataFrame(index=range(count_clasters), columns=[frequent_words_count, total_count_words] )
data_clasters_common_words[frequent_words_count] = words_info
data_clasters_common_words[total_count_words] = count_words
# data_clasters_common_words = pd.DataFrame([words_info, count_words])
data_clasters_common_words.tail(2)

,Частые слова и их количество,Общее количество слов
2064,"[(установка, 3), (бурильный, 3)]",6
2065,"[(установка, 4), (холодильный, 4)]",8


In [ ]:
# pd_clasters_common_words = pd.DataFrame(clasters_common_words)
# pd_clasters_common_words

,0
Группа продукции_ordinal,
0.0,"([(автоматический, 2), (установка, 2), (жидкос..."
1.0,"([(автоматический, 25), (устройство, 25), (упр..."
2.0,"([(автоматический, 3), (устройство, 3), (управ..."
3.0,"([(автопогрузчик, 6)], 6)"
4.0,"([(фара, 9), (огонь, 6), (освещение, 3), (задн..."
...,...
2061.0,"([(средство, 5), (учебный, 5), (имитационный, ..."
2062.0,"([(станок, 7), (бурение, 7), (скважина, 7), (г..."
2063.0,"([(взрывоопасный, 4), (оборудование, 3), (элек..."


In [ ]:
pd_clasters_common_words = pd.DataFrame(clasters_common_words)
pd_clasters_common_words.to_excel(f'{dir_data_out}/clasters_common_words.xlsx')

In [ ]:
# data_clasters_group = pd.DataFrame()

# data_clasters_group['Кластер'] = range(len(clasters_common_words))
# data_clasters_group['Наиболее встречаемые слова'] = clasters_common_words
# data_clasters_group['Наиболее встречаемые слова'] = data_clasters_group['Наиболее встречаемые слова'].fillna('-')
# data_clasters_group.to_excel(f'{dir_data_out}/data_clasters_group.xlsx', header=0)
# data_clasters_group.tail()

,Кластер,Наиболее встречаемые слова
2061,2061,"([(средство, 5), (учебный, 5), (имитационный, ..."
2062,2062,"([(станок, 7), (бурение, 7), (скважина, 7), (г..."
2063,2063,"([(взрывоопасный, 4), (оборудование, 3), (элек..."
2064,2064,"([(установка, 3), (бурильный, 3)], 6)"
2065,2065,"([(установка, 4), (холодильный, 4)], 8)"


In [ ]:
data_clasters_group.to_excel('data_clasters_group.exls')

In [ ]:
data_lemm_out.iloc[1][column_name]

'клапан противопожарный нормально открытый клапан противопожарный нормально закрытый люк дымовой'

In [ ]:
product_name = data_lemm_out.iloc[1][column_name]

In [ ]:
product_name.split(' ')

['клапан',
 'противопожарный',
 'нормально',
 'открытый',
 'клапан',
 'противопожарный',
 'нормально',
 'закрытый',
 'люк',
 'дымовой']

In [ ]:
data_clasters_common_words[[frequent_words_count]]

,Частые слова и их количество
0,"[(автоматический, 2), (установка, 2), (жидкост..."
1,"[(автоматический, 25), (устройство, 25), (упра..."
2,"[(автоматический, 3), (устройство, 3), (управл..."
3,"[(автопогрузчик, 6)]"
4,"[(фара, 9), (огонь, 6), (освещение, 3), (задни..."
...,...
2061,"[(средство, 5), (учебный, 5), (имитационный, 5)]"
2062,"[(станок, 7), (бурение, 7), (скважина, 7), (го..."
2063,"[(взрывоопасный, 4), (оборудование, 3), (элект..."
2064,"[(установка, 3), (бурильный, 3)]"


In [ ]:
# data_clasters_common_words[frequent_words_count]

0       [(автоматический, 2), (установка, 2), (жидкост...
1       [(автоматический, 25), (устройство, 25), (упра...
2       [(автоматический, 3), (устройство, 3), (управл...
3                                    [(автопогрузчик, 6)]
4       [(фара, 9), (огонь, 6), (освещение, 3), (задни...
                              ...                        
2061     [(средство, 5), (учебный, 5), (имитационный, 5)]
2062    [(станок, 7), (бурение, 7), (скважина, 7), (го...
2063    [(взрывоопасный, 4), (оборудование, 3), (элект...
2064                     [(установка, 3), (бурильный, 3)]
2065                   [(установка, 4), (холодильный, 4)]
Name: Частые слова и их количество, Length: 2066, dtype: object

In [ ]:
data_clasters_common_words[:][[frequent_words_count]]

,Частые слова и их количество
0,"[(автоматический, 2), (установка, 2), (жидкост..."
1,"[(автоматический, 25), (устройство, 25), (упра..."
2,"[(автоматический, 3), (устройство, 3), (управл..."
3,"[(автопогрузчик, 6)]"
4,"[(фара, 9), (огонь, 6), (освещение, 3), (задни..."
...,...
2061,"[(средство, 5), (учебный, 5), (имитационный, 5)]"
2062,"[(станок, 7), (бурение, 7), (скважина, 7), (го..."
2063,"[(взрывоопасный, 4), (оборудование, 3), (элект..."
2064,"[(установка, 3), (бурильный, 3)]"


In [ ]:
data_clasters_common_words[2:2][[frequent_words_count]]

,Частые слова и их количество


In [ ]:
# words = [a_tuple[1] for a_tuple in data_clasters_common_words[frequent_words_count]]
# count_words = [a_tuple[0] for a_tuple in data_clasters_common_words[frequent_words_count]]

# l = [(1,2), (3,4), (8,9)]
# list(zip(*l))

# list_words_count = list(zip(*data_clasters_common_words[:][[frequent_words_count]]))


d1 = map(list, zip(*data_clasters_common_words[1:1][[frequent_words_count]])) 


# asdfsdf = pd.DataFrame([list_words_count])
list_words_count
print(d1)

ValueError: ignored

In [ ]:
# data_clasters_common_words.loc[2:2, data_clasters_common_words[frequent_words_count] == 

# list(*data_clasters_common_words.loc[2:2, frequent_words_count])

# a, b = zip(*list(*data_clasters_common_words.loc[2:2, frequent_words_count]))
# c = list(a)
# c



words, count_words = map(list,zip(*list(*data_clasters_common_words.loc[1:1, frequent_words_count])))
words
# count_words
# data = list(filter(lambda x: x[0] == name, contacts))


['автоматический',
 'устройство',
 'управление',
 'бытовой',
 'электрический',
 'прибор',
 'работать',
 'автономно']

In [ ]:
words, count_words = map(list, zip(*data_clasters_common_words[frequent_words_count]))
words

ValueError: ignored

In [ ]:
words, count_words = map(list,zip(*list(*data_clasters_common_words.loc[1:1, frequent_words_count])))
words

In [ ]:
clasters_common_words.tail(2)

Группа продукции_ordinal
2064.0      ([(установка, 3), (бурильный, 3)], 6)
2065.0    ([(установка, 4), (холодильный, 4)], 8)
dtype: object

In [ ]:
data_clasters_common_words[:1]['Общее количество слов']

0    10
Name: Общее количество слов, dtype: int64

In [ ]:
def get_metrics(product_name, clasters_common_words): # Брать 2 вариант
  list_name_words = pd.unique(product_name.split(' '))
  for word in list_name_words:  
    word_name_count = np.sum(list_name_words == word) # !!!
    name_len = len(product_name) # !!! # Необязателен, так, как не изменяется для одной и той же фразы

    print(range(count_clasters))

    k = []
    for id_claster in range(count_clasters): # Внутри кластера суммировать или умножать - подумать
      
      claster_len = data_clasters_common_words[:id_claster][total_count_words] # !!!
      # Tfidf - поискать наименование метрики - ??? 

      words, count_words = map(list,zip(*list(*data_clasters_common_words.loc[1:1, frequent_words_count])))
      if word in words:
        id_word = words.index(element) 
        word_claster_count = count_words[id_word] # !!!

      print(f'Something {str(claster_len)}')

      k =  word_claster_count / (word_name_count * claster_len) # !!!!! Подумать, как правильно !!!


get_metrics(product_name, clasters_common_words)

Выходные данные были обрезаны до нескольких последних строк (5000).
4       76
      ... 
346     54
347     16
348      4
349     12
350     10
Name: Общее количество слов, Length: 351, dtype: int64
Something 0       10
1      200
2       51
3        6
4       76
      ... 
347     16
348      4
349     12
350     10
351     58
Name: Общее количество слов, Length: 352, dtype: int64
Something 0       10
1      200
2       51
3        6
4       76
      ... 
348      4
349     12
350     10
351     58
352    132
Name: Общее количество слов, Length: 353, dtype: int64
Something 0       10
1      200
2       51
3        6
4       76
      ... 
349     12
350     10
351     58
352    132
353      4
Name: Общее количество слов, Length: 354, dtype: int64
Something 0       10
1      200
2       51
3        6
4       76
      ... 
350     10
351     58
352    132
353      4
354     10
Name: Общее количество слов, Length: 355, dtype: int64
Something 0       10
1      200
2       51
3        6
4 

KeyboardInterrupt: ignored

In [ ]:
def get_metrics(product_name, clasters_common_words): # Брать 2 вариант
  list_name_words = pd.unique(product_name.split(' '))

  list_k = []
  for id_claster in range(count_clasters): # Внутри кластера суммировать или умножать - подумать    
    # print(range(count_clasters))

    claster_len = data_clasters_common_words[:id_claster][total_count_words] # !!!
    # Tfidf - поискать наименование метрики - ??? 

    # print(f'Claster id: {str(id_claster)}')
    k = 0
    for word in list_name_words:  
      word_name_count = np.sum(list_name_words == word) # !!!
      name_len = len(product_name) # !!! # Необязателен, так, как не изменяется для одной и той же фразы

      claster_words, claster_count_words = map(list,zip(*list(*data_clasters_common_words.loc[1:1, frequent_words_count]))) # Неудачные названия: claster_count_words, word_claster_count
      if word in claster_words:
        id_word = claster_words.index(element) 
        word_claster_count = claster_count_words[id_word] # !!!

        k +=  word_claster_count / (word_name_count * claster_len) # !!!!! Подумать, как правильно !!!

    list_k.append(k)

  return list_k


list_k = get_metrics(product_name, clasters_common_words)
list_k

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
len(list_k)

2066

In [ ]:
data_k = pd.DataFrame(list_k, index=range(count_clasters), columns=['Коэф. k'])
data_k

,Коэф. k
0,0
1,0
2,0
3,0
4,0
...,...
2061,0
2062,0
2063,0
2064,0


In [ ]:
# Добавить корректирующие коэффициенты в зависимости от длины наименования товара и кластеров

In [ ]:
 pd.DataFrame(index=range(count_clasters), columns=[frequent_words_count, total_count_words] )